In [1]:
import pandas as pd
import numpy as np
import operator
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [67]:
quant_df = pd.read_csv('analysisfile.csv')
face_df = pd.read_csv('face.csv')
survey_df = pd.read_csv('survey.csv')
df4 = pd.read_csv('df4.csv')

In [68]:
# mapped emotions: happiness, sadness, positive_emotion, negative_emotion

# -1 (negative, sadness) to 1 (positive, happiness)
# very unprofessional typed-by-hand values :P
emotion_mapping = {'amazement': 0.5 , 'joy': 1, 'sadness': -1, 'interest': 0.5, 
        'rage': -1, 'surprise': 0.3, 'serenity': 1, 'grief': -1,
        'fear': -0.75, 'anger': -1, 'ecstasy': 0.75, 'terror': -1,
        'trust': 0.75, 'boredom': -0.25, 'distraction': 0, 
        'acceptance': 0.5, 'disgust': -0.25, 'anticipation': 0.25}
# emotion_mapping = {'amazement': 'happiness' , 'joy': 'happiness', 'sadness': 'sadness', 'interest': 'positive_emotion', 
#         'rage': 'negative_emotion', 'surprise': 'positive_emotion', 'serenity': 'happiness', 'grief': 'sadness',
#         'fear': 'negative_emotion', 'anger': 'negative_emotion', 'ecstasy': 'positive_emotion', 'terror': 'negative_emotion',
#         'trust': 'positive_emotion', 'boredom': 'negative_emotion', 'distraction': 'negative_emotion', 
#         'acceptance': 'positive_emotion', 'disgust': 'negative_emotion', 'anticipation': 'positive_emotion'}

#df4['emotion_label'].unique()

#df4['avg_emotion_value'] = df4.apply(lambda x: emotion_mapping[x['emotion_label']], axis=1)

new_df = df4.copy()

new_df['uses_filter'] = df4.apply(lambda x: 0 if x['image_filter'] == 'Normal' else 1, axis=1)

new_df = new_df.drop(['emotion_label', 'emotion_score', 'image_filter'], axis=1)

new_df = new_df.groupby(by='user_id').mean()

new_df['user_id'] = new_df.index

new_df['user_id'] = new_df.apply(lambda x: int(x['user_id']), axis=1)

new_df.reset_index(drop=True)

new_df.index = range(len(new_df.index))

In [69]:
new_df = new_df.join(survey_df)

new_df = new_df.drop(['A_2', 'N_1', 'P_1', 'E_1', 'A_1',
       'H_1', 'M_1', 'R_1', 'M_2', 'E_2', 'LON', 'H_2', 'P_2', 'N_2', 'A_3',
       'N_3', 'E_3', 'H_3', 'R_2', 'M_3', 'R_3', 'P_3', 'HAP','network_id', 'P', 'E', 'R', 'M', 'A', 'start_q', 'end_q', 'private_account'], axis=1)

new_df.dropna(how='any', inplace=True)

X = new_df[['user_posted_photos', 'user_follows', 'user_followed_by', 'no_of_faces', 'imagecount']].copy()
X['x1'] = X['user_posted_photos'] * X['user_follows']
X['x2'] = X['user_followed_by'] * X['user_followed_by']

y = new_df['PERMA'].copy()

In [70]:
from sklearn.linear_model import LogisticRegression, Lasso, Ridge
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

lm = LogisticRegression()

#predicted = cross_val_predict(lr, X_train, y_test, cv=10)

predicted = cross_val_predict(LogisticRegression(), X, y, cv=15)

accuracy = metrics.accuracy_score(y, predicted)

print('Accuracy: ', accuracy)

# Show metrics
#print('Coefficients: \n', lm.coef_)
# The mean squared error
# print("Mean squared error: %.2f"
#       % mean_squared_error(y, y_predict))
# # Explained variance score: 1 is perfect prediction
# print('Variance score: %.2f' % r2_score(y, y_predict))

('Accuracy: ', 0.25675675675675674)
